In [7]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from rdkit.DataStructs import ConvertToNumpyArray
import torch
from torch_geometric.data import Data
import deepchem as dc
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# --- 1. RDKit Featurization Functions ---

def one_hot_encode(value, allowed_set):
    if value not in allowed_set:
        value = allowed_set[-1]
    return [int(value == s) for s in allowed_set]

def get_atom_features(atom):
    """More robust one-hot atom features."""
    ATOM_SYMBOLS = ['C', 'N', 'O', 'F', 'P', 'S', 'Cl', 'Br', 'I', 'other']
    DEGREES = [0, 1, 2, 3, 4, 5, 'other']
    HYBRIDIZATIONS = [
        Chem.rdchem.HybridizationType.SP, Chem.rdchem.HybridizationType.SP2,
        Chem.rdchem.HybridizationType.SP3, Chem.rdchem.HybridizationType.SP3D,
        Chem.rdchem.HybridizationType.SP3D2, 'other'
    ]
    FORMAL_CHARGES = [-2, -1, 0, 1, 2, 'other']
    TOTAL_HS = [0, 1, 2, 3, 4, 'other']
    
    features = (
        one_hot_encode(atom.GetSymbol(), ATOM_SYMBOLS) +
        one_hot_encode(atom.GetDegree(), DEGREES) +
        one_hot_encode(atom.GetHybridization(), HYBRIDIZATIONS) +
        one_hot_encode(atom.GetFormalCharge(), FORMAL_CHARGES) +
        one_hot_encode(atom.GetTotalNumHs(), TOTAL_HS) +
        [int(atom.GetIsAromatic())] +
        [int(atom.IsInRing())]
    )
    return features

def featurize_graph(mol):
    """Converts an RDKit Mol object into graph node/edge features."""
    try:
        mol = Chem.AddHs(mol)
        node_features = [get_atom_features(atom) for atom in mol.GetAtoms()]
        if not node_features:
            return None
        
        edge_indices = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            edge_indices.append((i, j))
            edge_indices.append((j, i))

        node_features_np = np.array(node_features, dtype=np.float32)
        
        if edge_indices:
            edge_index_np = np.array(edge_indices, dtype=np.int64).T
        else:
            edge_index_np = np.array([], dtype=np.int64).reshape(2, 0)
            
        return {
            'node_features': torch.tensor(node_features_np, dtype=torch.float),
            'edge_index': torch.tensor(edge_index_np, dtype=torch.long)
        }
    except Exception as e:
        # print(f"Warning: RDKit error featurizing graph: {e}")
        return None

def get_fingerprint_features(mol):
    """Generates Morgan (ECFP) fingerprints."""
    try:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
        fp_np = np.zeros((1,), dtype=np.int8)
        ConvertToNumpyArray(fp, fp_np)
        return torch.tensor(fp_np.astype(np.float32), dtype=torch.float)
    except Exception as e:
        return None

def get_descriptor_features(mol):
    """Generates a panel of RDKit descriptors."""
    descriptor_names = [
        'MolWt', 'MolLogP', 'NumHDonors', 'NumHAcceptors',
        'TPSA', 'NumRotatableBonds', 'FpDensityMorgan1', 'FractionCSP3'
    ]
    features = []
    for name in descriptor_names:
        try:
            desc_val = getattr(Descriptors, name)(mol)
            features.append(desc_val)
        except:
            features.append(np.nan)
    return features # Return a list for now, we'll numpy/tensor it later

# --- 2. Main Data Loading and Processing ---

print("📥 Loading Tox21 dataset from DeepChem...")
# tasks, datasets, transformers = dc.molnet.load_tox21(featurizer="Raw", data_dir=".", save_dir=".")
# train_dataset, valid_dataset, test_dataset = datasets

# # Convert to pandas DataFrames
# train_df = train_dataset.to_dataframe()
# valid_df = valid_dataset.to_dataframe()
# test_df = test_dataset.to_dataframe()

# # map tasks to column names such as tasks[0] is y1 column, replace it with tasks[0]
# for i, task in enumerate(tasks):
#     train_df.rename(columns={f'y{i+1}': task}, inplace=True)
#     valid_df.rename(columns={f'y{i+1}': task}, inplace=True)
#     test_df.rename(columns={f'y{i+1}': task}, inplace=True)

train_df = pd.read_csv('tox21_processed/train_raw.csv')
valid_df = pd.read_csv('tox21_processed/valid_raw.csv')
test_df = pd.read_csv('tox21_processed/test_raw.csv')

print(f"Original sizes: Train={len(train_df)}, Valid={len(valid_df)}, Test={len(test_df)}")

# Define all column names
LABEL_COLUMNS = [
    'NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 
    'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 
    'SR-HSE', 'SR-MMP', 'SR-p53'
]
# This assumes the 'w' columns are named w1, w2, etc. in the dataframe
# Let's check the columns and create the list dynamically
weight_columns = [col for col in train_df.columns if col.startswith('w') and col[1:].isdigit()]
if len(weight_columns) != 12:
    print(f"Warning: Expected 12 weight columns ('w1'...'w12'), found {len(weight_columns)}. Using them anyway.")
WEIGHT_COLUMNS = weight_columns

SMILES_COLUMN = 'ids'
DESCRIPTOR_COLUMNS = [
    'MolWt', 'MolLogP', 'NumHDonors', 'NumHAcceptors',
    'TPSA', 'NumRotatableBonds', 'FpDensityMorgan1', 'FractionCSP3'
]

# --- 3. Pre-calculate Descriptors and Handle Scaling ---
# We do this *before* the main loop to handle scaling and imputing correctly
print("🔬 Pre-calculating descriptors for scaling...")

def add_descriptors(df):
    desc_data = []
    for smiles in tqdm(df[SMILES_COLUMN], desc="Calculating descriptors"):
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            desc_data.append(get_descriptor_features(mol))
        else:
            desc_data.append([np.nan] * len(DESCRIPTOR_COLUMNS))
    return pd.concat([df, pd.DataFrame(desc_data, columns=DESCRIPTOR_COLUMNS, index=df.index)], axis=1)

train_df = add_descriptors(train_df)
valid_df = add_descriptors(valid_df)
test_df = add_descriptors(test_df)

# Impute and Scale Descriptors
print("⚖️ Imputing and Scaling descriptors...")
imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()

# Fit on training data ONLY
imputer.fit(train_df[DESCRIPTOR_COLUMNS])
train_df[DESCRIPTOR_COLUMNS] = imputer.transform(train_df[DESCRIPTOR_COLUMNS])

scaler.fit(train_df[DESCRIPTOR_COLUMNS])
train_df[DESCRIPTOR_COLUMNS] = scaler.transform(train_df[DESCRIPTOR_COLUMNS])

# Transform valid and test data
valid_df[DESCRIPTOR_COLUMNS] = imputer.transform(valid_df[DESCRIPTOR_COLUMNS])
valid_df[DESCRIPTOR_COLUMNS] = scaler.transform(valid_df[DESCRIPTOR_COLUMNS])

test_df[DESCRIPTOR_COLUMNS] = imputer.transform(test_df[DESCRIPTOR_COLUMNS])
test_df[DESCRIPTOR_COLUMNS] = scaler.transform(test_df[DESCRIPTOR_COLUMNS])

print("✅ Scaling and Imputing complete.")

# --- 4. Main Featurization Loop ---

def create_data_list(df):
    """
    Processes a DataFrame and returns a list of PyG Data objects.
    This is now fast and efficient.
    """
    data_list = []
    
    # Use iterrows() to handle column names with special characters ('-')
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Creating Data objects"):
        smiles = row[SMILES_COLUMN]
        mol = Chem.MolFromSmiles(smiles)
        
        if mol is None:
            continue
        
        # 1. Get Graph Features
        graph_features = featurize_graph(mol)
        if graph_features is None:
            continue
            
        # 2. Get Fingerprint Features
        fp_features = get_fingerprint_features(mol)
        if fp_features is None:
            continue
            
        # 3. Get Pre-scaled Descriptor Features
        # Access by list/string name, not attribute
        desc_features = list(row[DESCRIPTOR_COLUMNS])
        desc_tensor = torch.tensor(desc_features, dtype=torch.float)

        # 4. Get Labels
        # Access by list/string name, not attribute
        labels = list(row[LABEL_COLUMNS])
        labels_tensor = torch.tensor(labels, dtype=torch.float).reshape(1, 12)

        # 5. Get Weights (Fix for Bug 1)
        # Access by list/string name, not attribute
        weights = list(row[WEIGHT_COLUMNS])
        weights_tensor = torch.tensor(weights, dtype=torch.float).reshape(1, 12)

        # 6. Create the "Early Fusion" Data Object
        data = Data(
            x=graph_features['node_features'],
            edge_index=graph_features['edge_index'],
            y=labels_tensor,
            w=weights_tensor,  # Store the 12 weights
            fp=fp_features.reshape(1, -1), # Store the fingerprint
            desc=desc_tensor.reshape(1, -1) # Store the descriptors
        )
        data_list.append(data)
        
    return data_list

# Process all datasets
print("\n--- Processing Training Data ---")
train_data_list = create_data_list(train_df)
print(f"\n--- Processing Validation Data ---")
valid_data_list = create_data_list(valid_df)
print(f"\n--- Processing Test Data ---")
test_data_list = create_data_list(test_df)

print("\n--- Featurization Summary ---")
print(f"Processed Train Graphs: {len(train_data_list)}")
print(f"Processed Valid Graphs: {len(valid_data_list)}")
print(f"Processed Test Graphs:  {len(test_data_list)}")

print("\nSample Data Object (from training set):")
if train_data_list:
    print(train_data_list[0])
    print("Notice it now contains 'y', 'w', 'fp', and 'desc' keys!")

# --- 5. Save the Processed Data ---
OUTPUT_DIR = "processed_fusion_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"\n💾 Saving processed data to '{OUTPUT_DIR}'...")
torch.save(train_data_list, os.path.join(OUTPUT_DIR, "train_data.pt"))
torch.save(valid_data_list, os.path.join(OUTPUT_DIR, "valid_data.pt"))
torch.save(test_data_list, os.path.join(OUTPUT_DIR, "test_data.pt"))

print("✅ All processing complete and saved!")

📥 Loading Tox21 dataset from DeepChem...
Original sizes: Train=6258, Valid=782, Test=783
🔬 Pre-calculating descriptors for scaling...


Calculating descriptors: 100%|██████████| 783/783 [00:00<00:00, 1161.78it/s]


⚖️ Imputing and Scaling descriptors...
✅ Scaling and Imputing complete.

--- Processing Training Data ---


Creating Data objects:   0%|          | 0/6258 [00:00<?, ?it/s][17:27:36] DEPRECATION WARNING: please use MorganGenerator
[17:27:36] DEPRECATION WARNING: please use MorganGenerator
[17:27:36] DEPRECATION WARNING: please use MorganGenerator
[17:27:36] DEPRECATION WARNING: please use MorganGenerator
[17:27:36] DEPRECATION WARNING: please use MorganGenerator
[17:27:36] DEPRECATION WARNING: please use MorganGenerator
[17:27:36] DEPRECATION WARNING: please use MorganGenerator
[17:27:36] DEPRECATION WARNING: please use MorganGenerator
[17:27:36] DEPRECATION WARNING: please use MorganGenerator
[17:27:36] DEPRECATION WARNING: please use MorganGenerator
[17:27:36] DEPRECATION WARNING: please use MorganGenerator
[17:27:36] DEPRECATION WARNING: please use MorganGenerator
[17:27:36] DEPRECATION WARNING: please use MorganGenerator
[17:27:36] DEPRECATION WARNING: please use MorganGenerator
[17:27:36] DEPRECATION WARNING: please use MorganGenerator
[17:27:36] DEPRECATION WARNING: please use MorganGen


--- Processing Validation Data ---


Creating Data objects:   0%|          | 0/782 [00:00<?, ?it/s][17:27:51] DEPRECATION WARNING: please use MorganGenerator
[17:27:51] DEPRECATION WARNING: please use MorganGenerator
[17:27:51] DEPRECATION WARNING: please use MorganGenerator
[17:27:51] DEPRECATION WARNING: please use MorganGenerator
[17:27:51] DEPRECATION WARNING: please use MorganGenerator
[17:27:51] DEPRECATION WARNING: please use MorganGenerator
[17:27:51] DEPRECATION WARNING: please use MorganGenerator
[17:27:51] DEPRECATION WARNING: please use MorganGenerator
[17:27:51] DEPRECATION WARNING: please use MorganGenerator
[17:27:51] DEPRECATION WARNING: please use MorganGenerator
[17:27:51] DEPRECATION WARNING: please use MorganGenerator
[17:27:51] DEPRECATION WARNING: please use MorganGenerator
[17:27:51] DEPRECATION WARNING: please use MorganGenerator
[17:27:51] DEPRECATION WARNING: please use MorganGenerator
[17:27:51] DEPRECATION WARNING: please use MorganGenerator
[17:27:51] DEPRECATION WARNING: please use MorganGene


--- Processing Test Data ---


Creating Data objects:   0%|          | 0/783 [00:00<?, ?it/s][17:27:53] DEPRECATION WARNING: please use MorganGenerator
[17:27:53] DEPRECATION WARNING: please use MorganGenerator
[17:27:53] DEPRECATION WARNING: please use MorganGenerator
[17:27:53] DEPRECATION WARNING: please use MorganGenerator
[17:27:53] DEPRECATION WARNING: please use MorganGenerator
[17:27:53] DEPRECATION WARNING: please use MorganGenerator
[17:27:53] DEPRECATION WARNING: please use MorganGenerator
[17:27:53] DEPRECATION WARNING: please use MorganGenerator
[17:27:53] DEPRECATION WARNING: please use MorganGenerator
[17:27:53] DEPRECATION WARNING: please use MorganGenerator
[17:27:53] DEPRECATION WARNING: please use MorganGenerator
[17:27:53] DEPRECATION WARNING: please use MorganGenerator
[17:27:53] DEPRECATION WARNING: please use MorganGenerator
[17:27:53] DEPRECATION WARNING: please use MorganGenerator
[17:27:53] DEPRECATION WARNING: please use MorganGenerator
[17:27:53] DEPRECATION WARNING: please use MorganGene


--- Featurization Summary ---
Processed Train Graphs: 6258
Processed Valid Graphs: 782
Processed Test Graphs:  783

Sample Data Object (from training set):
Data(x=[19, 37], edge_index=[2, 36], y=[1, 12], w=[1, 12], fp=[1, 2048], desc=[1, 8])
Notice it now contains 'y', 'w', 'fp', and 'desc' keys!

💾 Saving processed data to 'processed_fusion_data'...
✅ All processing complete and saved!


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv, global_mean_pool

class EarlyFusionModel(nn.Module):
    """
    A GNN-based model that fuses three types of input features:
    1. Graph features (nodes, edges)
    2. Fingerprint features (e.g., Morgan/ECFP)
    3. Descriptor features (e.g., PhysChem properties)
    """
    def __init__(self, 
                 node_feat_dim, 
                 fp_feat_dim, 
                 desc_feat_dim,
                 graph_hidden_dim=128,
                 graph_out_dim=128,
                 fp_out_dim=128,
                 desc_out_dim=32,
                 num_tasks=12):
        """
        Initialize the model components.
        
        Args:
            node_feat_dim (int): Dimensionality of node features (e.g., 37)
            fp_feat_dim (int): Dimensionality of fingerprint (e.g., 2048)
            desc_feat_dim (int): Dimensionality of descriptors (e.g., 8)
            graph_hidden_dim (int): Hidden size for GNN layers
            graph_out_dim (int): Output size of the GNN branch
            fp_out_dim (int): Output size of the fingerprint branch
            desc_out_dim (int): Output size of the descriptor branch
            num_tasks (int): Number of output tasks (12 for Tox21)
        """
        super(EarlyFusionModel, self).__init__()
        
        # --- 1. Graph Branch ---
        # We'll use GATv2 (Graph Attention Network v2) as it's powerful
        # You could also use SAGEConv or GCNConv
        self.conv1 = GATv2Conv(node_feat_dim, graph_hidden_dim, heads=4, dropout=0.2)
        self.conv2 = GATv2Conv(graph_hidden_dim * 4, graph_hidden_dim * 2, heads=4, dropout=0.2)
        self.conv3 = GATv2Conv(graph_hidden_dim * 2 * 4, graph_out_dim, heads=1, concat=False, dropout=0.2)
        
        # --- 2. Fingerprint Branch ---
        # An MLP to process the flat fingerprint vector
        self.fp_branch = nn.Sequential(
            nn.Linear(fp_feat_dim, fp_feat_dim // 4),
            nn.BatchNorm1d(fp_feat_dim // 4),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(fp_feat_dim // 4, fp_out_dim),
        )
        
        # --- 3. Descriptor Branch ---
        # A simple MLP to process the (already scaled) descriptors
        self.desc_branch = nn.Sequential(
            nn.Linear(desc_feat_dim, desc_out_dim),
            nn.BatchNorm1d(desc_out_dim),
            nn.ReLU(),
        )
        
        # --- 4. Final Classifier Head ---
        # This takes the concatenated outputs from all three branches
        self.fusion_dim = graph_out_dim + fp_out_dim + desc_out_dim
        
        self.classifier = nn.Sequential(
            nn.Linear(self.fusion_dim, self.fusion_dim // 2),
            nn.BatchNorm1d(self.fusion_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(self.fusion_dim // 2, self.fusion_dim // 4),
            nn.BatchNorm1d(self.fusion_dim // 4),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.fusion_dim // 4, num_tasks) # Output 12 logits
        )

    def forward(self, data):
        """
        Forward pass of the model.
        
        Args:
            data (torch_geometric.data.Batch): A batch of Data objects
                - data.x: Node features [num_nodes_in_batch, node_feat_dim]
                - data.edge_index: Edge connectivity [2, num_edges_in_batch]
                - data.batch: Batch assignment vector [num_nodes_in_batch]
                - data.fp: Fingerprint features [batch_size, fp_feat_dim]
                - data.desc: Descriptor features [batch_size, desc_feat_dim]
        """
        
        # --- 1. Process Graph Branch ---
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        x = F.elu(self.conv1(x, edge_index))
        x = F.elu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        
        # Global mean pooling to get one vector per graph
        graph_out = global_mean_pool(x, batch) # [batch_size, graph_out_dim]
        
        # --- 2. Process Fingerprint Branch ---
        # data.fp is [batch_size, fp_feat_dim]
        fp_out = self.fp_branch(data.fp) # [batch_size, fp_out_dim]
        
        # --- 3. Process Descriptor Branch ---
        # data.desc is [batch_size, desc_feat_dim]
        desc_out = self.desc_branch(data.desc) # [batch_size, desc_out_dim]
        
        # --- 4. Fuse and Classify ---
        # Concatenate all feature vectors
        fused_features = torch.cat([graph_out, fp_out, desc_out], dim=1)
        
        # Pass through the final classifier
        logits = self.classifier(fused_features) # [batch_size, num_tasks]
        
        return logits

# --- Example Usage (for testing the file) ---
if __name__ == '__main__':
    #from build_fusion_dataset import get_atom_features
    #from rdkit import Chem
    
    # 1. Get feature dimensions from our featurizer
    # We create a dummy atom to check the feature length
    dummy_mol = Chem.MolFromSmiles('C')
    dummy_atom = dummy_mol.GetAtomWithIdx(0)
    NODE_DIM = len(get_atom_features(dummy_atom))
    FP_DIM = 2048
    DESC_DIM = 8 # From our 8 descriptors
    NUM_TASKS = 12

    print(f"Model Configuration:")
    print(f"  Node features:    {NODE_DIM}")
    print(f"  Fingerprint dim:  {FP_DIM}")
    print(f"  Descriptor dim:   {DESC_DIM}")
    print(f"  Output tasks:     {NUM_TASKS}")

    # 2. Instantiate the model
    model = EarlyFusionModel(
        node_feat_dim=NODE_DIM,
        fp_feat_dim=FP_DIM,
        desc_feat_dim=DESC_DIM,
        num_tasks=NUM_TASKS
    )
    
    print("\nModel architecture:")
    print(model)
    
    # 3. Test with a dummy Batch object (simulating a batch of 2 graphs)
    from torch_geometric.data import Data, Batch
    
    # Graph 1: 3 nodes, 2 edges
    data1 = Data(
        x=torch.randn(3, NODE_DIM),
        edge_index=torch.tensor([[0, 1], [1, 2]], dtype=torch.long).t(),
        fp=torch.randn(1, FP_DIM),
        desc=torch.randn(1, DESC_DIM)
    )
    # Graph 2: 4 nodes, 3 edges
    data2 = Data(
        x=torch.randn(4, NODE_DIM),
        edge_index=torch.tensor([[0, 1], [1, 2], [2, 3]], dtype=torch.long).t(),
        fp=torch.randn(1, FP_DIM),
        desc=torch.randn(1, DESC_DIM)
    )
    
    # Create a batch
    batch = Batch.from_data_list([data1, data2])
    print(f"\nTesting forward pass with dummy batch (batch_size=2)...")
    
    # Perform forward pass
    output_logits = model(batch)
    
    print(f"Output logits shape: {output_logits.shape}")
    assert output_logits.shape == (2, NUM_TASKS)
    print("✅ Forward pass successful!")


Model Configuration:
  Node features:    37
  Fingerprint dim:  2048
  Descriptor dim:   8
  Output tasks:     12

Model architecture:
EarlyFusionModel(
  (conv1): GATv2Conv(37, 128, heads=4)
  (conv2): GATv2Conv(512, 256, heads=4)
  (conv3): GATv2Conv(1024, 128, heads=1)
  (fp_branch): Sequential(
    (0): Linear(in_features=2048, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=512, out_features=128, bias=True)
  )
  (desc_branch): Sequential(
    (0): Linear(in_features=8, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=288, out_features=144, bias=True)
    (1): BatchNorm1d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=F

In [9]:
from torch_geometric.loader import DataLoader
import math
from tqdm import tqdm

# Training loop for the EarlyFusionModel using pre-built Data lists:
# Assumes the notebook has already defined:
#   - train_data_list, valid_data_list, test_data_list (lists of torch_geometric.data.Data)
#   - EarlyFusionModel class
#   - OUTPUT_DIR (or will create it here)
# This cell will create dataloaders, instantiate the model, train, validate, and save best model.

import torch.optim as optim

# Config
batch_size = 32
lr = 1e-4
weight_decay = 1e-5
num_epochs = 30
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs(OUTPUT_DIR, exist_ok=True)
best_val_loss = math.inf
best_path = os.path.join(OUTPUT_DIR, "best_early_fusion_model.pt")

# Infer dims from dataset examples (do not re-create featurizers)
sample = train_data_list[0]
node_feat_dim = sample.x.shape[1]
fp_feat_dim = sample.fp.shape[1]
desc_feat_dim = sample.desc.shape[1]
num_tasks = sample.y.shape[-1]

# Dataloaders
train_loader = DataLoader(train_data_list, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data_list, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_data_list,  batch_size=batch_size, shuffle=False)

# Model, optimizer, scheduler
model = EarlyFusionModel(
    node_feat_dim=node_feat_dim,
    fp_feat_dim=fp_feat_dim,
    desc_feat_dim=desc_feat_dim,
    num_tasks=num_tasks
).to(device)

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

# Training / validation helpers
def unpack_batch_targets(batch):
    # batch.y and batch.w should stack to shape [batch_size, num_tasks]
    y = batch.y.view(-1, num_tasks).to(device)
    w = batch.w.view(-1, num_tasks).to(device)
    return y, w

def train_one_epoch(epoch):
    model.train()
    running_loss = 0.0
    n_batches = 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch} [train]", leave=False)
    for batch in pbar:
        batch = batch.to(device)
        optimizer.zero_grad()
        logits = model(batch)                 # [B, num_tasks]
        y, w = unpack_batch_targets(batch)    # [B, num_tasks], [B, num_tasks]
        loss = torch.nn.functional.binary_cross_entropy_with_logits(logits, y, weight=w)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        n_batches += 1
        pbar.set_postfix(loss=running_loss / n_batches)
    return running_loss / max(1, n_batches)

def evaluate(loader, stage="valid"):
    model.eval()
    running_loss = 0.0
    n_batches = 0
    with torch.no_grad():
        pbar = tqdm(loader, desc=f"Eval [{stage}]", leave=False)
        for batch in pbar:
            batch = batch.to(device)
            logits = model(batch)
            y, w = unpack_batch_targets(batch)
            loss = torch.nn.functional.binary_cross_entropy_with_logits(logits, y, weight=w)
            running_loss += loss.item()
            n_batches += 1
            pbar.set_postfix(loss=running_loss / n_batches)
    return running_loss / max(1, n_batches)

# Main training loop
for epoch in range(1, num_epochs + 1):
    train_loss = train_one_epoch(epoch)
    val_loss = evaluate(valid_loader, stage="valid")

    scheduler.step(val_loss)

    print(f"Epoch {epoch:03d} | train_loss={train_loss:.4f} | val_loss={val_loss:.4f} | lr={optimizer.param_groups[0]['lr']:.2e}")

    # Save best
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_path)
        print(f"  Saved new best model (val_loss={val_loss:.4f}) -> {best_path}")

# Final test evaluation with best model
print("Loading best model and evaluating on test set...")
model.load_state_dict(torch.load(best_path, map_location=device))
test_loss = evaluate(test_loader, stage="test")
print(f"Final test loss: {test_loss:.4f}")

Epoch 001 | train_loss=1.1572 | val_loss=1.1837 | lr=1.00e-04
  Saved new best model (val_loss=1.1837) -> processed_fusion_data\best_early_fusion_model.pt


Epoch 002 | train_loss=1.0709 | val_loss=1.1437 | lr=1.00e-04
  Saved new best model (val_loss=1.1437) -> processed_fusion_data\best_early_fusion_model.pt


Epoch 003 | train_loss=1.0171 | val_loss=1.1124 | lr=1.00e-04
  Saved new best model (val_loss=1.1124) -> processed_fusion_data\best_early_fusion_model.pt


Epoch 004 | train_loss=0.9500 | val_loss=1.1025 | lr=1.00e-04
  Saved new best model (val_loss=1.1025) -> processed_fusion_data\best_early_fusion_model.pt


Epoch 005 | train_loss=0.8857 | val_loss=1.0827 | lr=1.00e-04
  Saved new best model (val_loss=1.0827) -> processed_fusion_data\best_early_fusion_model.pt


...

Epoch 011 | train_loss=0.5978 | val_loss=1.1163 | lr=5.00e-05


Epoch 012 | train_loss=0.5710 | val_loss=1.1535 | lr=5.00e-05


Epoch 00013: reducing learning rate of group 0 to 2.5000e-05.
Epoch 013 | train_loss=0.5559 | val_loss=1.1460 | lr=2.50e-05


Epoch 014 | train_loss=0.5370 | val_loss=1.1697 | lr=2.50e-05


KeyboardInterrupt: 

In [10]:
# Final test evaluation with best model
print("Loading best model and evaluating on test set...")
model.load_state_dict(torch.load(best_path, map_location=device))
test_loss = evaluate(test_loader, stage="test")
print(f"Final test loss: {test_loss:.4f}")

# calculate auc-roc scores
y_true = []
y_scores = []

for batch in test_loader:
    batch = batch.to(device)
    with torch.no_grad():
        logits = model(batch)
        probs = torch.sigmoid(logits).cpu().numpy()
    y_scores.append(probs)
    y_true.append(batch.y.view(-1, num_tasks).cpu().numpy())

y_true = np.vstack(y_true)
y_scores = np.vstack(y_scores)

from sklearn.metrics import roc_auc_score
auc_scores = []
for i in range(num_tasks):
    try:
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
    except ValueError:
        auc = float('nan')  # Handle cases where only one class is present
    auc_scores.append(auc)
print("\nAUC-ROC Scores per Task:")
for i, score in enumerate(auc_scores):
    print(f"  Task {i+1}: {score:.4f}")
    # Final test evaluation with best model

Loading best model and evaluating on test set...


Final test loss: 1.0365

AUC-ROC Scores per Task:
  Task 1: 0.7637
  Task 2: 0.7116
  Task 3: 0.7859
  Task 4: 0.7087
  Task 5: 0.6481
  Task 6: 0.6928
  Task 7: 0.6648
  Task 8: 0.6154
  Task 9: 0.7273
  Task 10: 0.7041
  Task 11: 0.7292
  Task 12: 0.7007


In [ ]:
# import os
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch_geometric.loader import DataLoader
# from sklearn.metrics import roc_auc_score
# from tqdm import tqdm

# # Import our custom model
# from model import EarlyFusionModel

# --- 1. Constants and Setup ---
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

DATA_DIR = "processed_fusion_data"
MODEL_SAVE_PATH = "best_fusion_model.pth"

# Hyperparameters
BATCH_SIZE = 64
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5
EPOCHS = 10
PATIENCE = 10  # For early stopping
NUM_TASKS = 12

# --- 2. Load Data and Create Loaders ---

print("Loading processed data...")
try:
    train_data = torch.load(os.path.join(DATA_DIR, "train_data.pt"))
    valid_data = torch.load(os.path.join(DATA_DIR, "valid_data.pt"))
    test_data = torch.load(os.path.join(DATA_DIR, "test_data.pt"))
except FileNotFoundError:
    print(f"Error: Processed data not found in '{DATA_DIR}'.")
    print("Please run 'build_fusion_dataset.py' first.")
    exit()

if not train_data:
    print("Error: Training data is empty.")
    exit()

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

print(f"Data loaded: {len(train_data)} train, {len(valid_data)} valid, {len(test_data)} test samples.")

# --- 3. Initialize Model and Optimizer ---

# Dynamically get feature dimensions from the first data object
first_data = train_data[0]
NODE_DIM = first_data.x.shape[1]
FP_DIM = first_data.fp.shape[1]
DESC_DIM = first_data.desc.shape[1]

print(f"Feature Dims: Node={NODE_DIM}, FP={FP_DIM}, Desc={DESC_DIM}")

model = EarlyFusionModel(
    node_feat_dim=NODE_DIM,
    fp_feat_dim=FP_DIM,
    desc_feat_dim=DESC_DIM,
    num_tasks=NUM_TASKS
).to(DEVICE)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Define the LOSS FUNCTION (using weights!)
# We use 'none' reduction to get per-element losses, which we then multiply by our weights.
loss_fn = nn.BCEWithLogitsLoss(reduction='none')

# --- 4. Training and Evaluation Functions ---

def train_epoch(model, loader, loss_fn, optimizer):
    model.train()
    total_loss = 0
    for batch in tqdm(loader, desc="Training", leave=False):
        batch = batch.to(DEVICE)
        
        # Forward pass
        logits = model(batch)
        
        # Calculate weighted loss
        y_true = batch.y
        weights = batch.w
        
        # raw_loss shape: [batch_size, num_tasks]
        raw_loss = loss_fn(logits, y_true)
        
        # Apply weights
        weighted_loss = raw_loss * weights
        
        # Calculate mean loss, ignoring 0-weight entries
        # We sum all weighted losses and divide by the sum of all weights
        # (This correctly handles missing labels)
        final_loss = weighted_loss.sum() / weights.sum()

        # Backpropagation
        optimizer.zero_grad()
        final_loss.backward()
        optimizer.step()
        
        total_loss += final_loss.item() * batch.num_graphs
    
    return total_loss / len(loader.dataset)


@torch.no_grad()
def eval_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    all_weights = []
    
    for batch in tqdm(loader, desc="Evaluating", leave=False):
        batch = batch.to(DEVICE)
        
        # Forward pass
        logits = model(batch)
        preds = torch.sigmoid(logits)
        
        # Collect predictions, labels, and weights
        all_preds.append(preds.cpu().numpy())
        all_labels.append(batch.y.cpu().numpy())
        all_weights.append(batch.w.cpu().numpy())
        
    # Concatenate all results
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    all_weights = np.concatenate(all_weights, axis=0)
    
    # Calculate per-task ROC-AUC
    task_aucs = []
    for i in range(NUM_TASKS):
        task_labels = all_labels[:, i]
        task_preds = all_preds[:, i]
        task_weights = all_weights[:, i]
        
        # Filter for valid labels (where weight > 0)
        valid_indices = task_weights > 0
        
        # Check if we have enough valid samples (at least one of each class)
        if np.sum(valid_indices) > 0 and len(np.unique(task_labels[valid_indices])) == 2:
            try:
                auc = roc_auc_score(task_labels[valid_indices], task_preds[valid_indices])
                task_aucs.append(auc)
            except ValueError:
                task_aucs.append(np.nan) # Should not happen, but for safety
        else:
            task_aucs.append(np.nan) # Not enough data to calculate AUC
            
    # Calculate mean AUC, ignoring tasks with NaN
    mean_auc = np.nanmean(task_aucs)
    return mean_auc

# --- 5. Main Training Loop ---

print("\n--- Starting Model Training ---")

best_valid_auc = 0.0
epochs_no_improve = 0

for epoch in range(1, EPOCHS + 1):
    train_loss = train_epoch(model, train_loader, loss_fn, optimizer)
    valid_auc = eval_model(model, valid_loader)
    
    print(f"Epoch {epoch:03d}/{EPOCHS} | Train Loss: {train_loss:.4f} | Valid AUC: {valid_auc:.4f}")
    
    # Check for improvement
    if valid_auc > best_valid_auc:
        best_valid_auc = valid_auc
        epochs_no_improve = 0
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"🎉 New best model saved to {MODEL_SAVE_PATH} (AUC: {best_valid_auc:.4f})")
    else:
        epochs_no_improve += 1
        
    # Check for early stopping
    if epochs_no_improve >= PATIENCE:
        print(f"Early stopping triggered after {PATIENCE} epochs with no improvement.")
        break

# --- 6. Final Evaluation ---

print("\n--- Training Complete ---")

# Load the best model
print(f"Loading best model from {MODEL_SAVE_PATH} for final test.")
model.load_state_dict(torch.load(MODEL_SAVE_PATH))

test_auc = eval_model(model, test_loader)

print("\n--- Final Results ---")
print(f"Best Validation AUC: {best_valid_auc:.4f}")
print(f"Final Test AUC:      {test_auc:.4f}")


Using device: cpu
Loading processed data...
Data loaded: 6258 train, 782 valid, 783 test samples.
Feature Dims: Node=37, FP=2048, Desc=8

--- Starting Model Training ---


Epoch 001/10 | Train Loss: 0.6889 | Valid AUC: 0.6288
🎉 New best model saved to best_fusion_model.pth (AUC: 0.6288)


Epoch 002/10 | Train Loss: 0.6459 | Valid AUC: 0.6697
🎉 New best model saved to best_fusion_model.pth (AUC: 0.6697)


Epoch 003/10 | Train Loss: 0.6175 | Valid AUC: 0.7016
🎉 New best model saved to best_fusion_model.pth (AUC: 0.7016)


Training:  28%|██▊       | 27/98 [02:47<09:53,  8.36s/it]